## CIS5560: PySpark Pipeline Text Sentiment Analysis in Databricks

### by Team 4 (Uche, Raymond, Tofunmi and Sweta) edited on 05/15/2020
Tested in Runtime 6.5 (Spark 2.4.5/2.4.0 Scala 2.11) of Databricks CE

## Text Analysis
In this project, we created a classification model that performs sentiment analysis of reviews of different businesses.
### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

## Steps to download dataset and do some data engineering (Cleaning up dataset) before importing into databricks

all dataset engineering were done in Jupyter Notebook before importing into databricks

dataset link: https://www.kaggle.com/darshank2019/review#yelp_academic_dataset_review.csv

download dataset and using a Jupyter Notebook(we used google colab), we accessed the dataset with total rows = 6685900

we took a slice of the full dataset of the first 1500000 rows and used that as our full dataset.

we removed the inverted commas and the letter "b" present in all rows (data cleaning)

we converted the alphanumeric values in the user_id, review_id, & business_id to numeric values

we tried to drop rows wit missing values and counted the total number of rows again and it was still 1500000.

we created a subset of our cleaned dataset named df_ml_csv with 120000 rows which we used for both Azure ML & Databricks

NOTE: the .py & .ipynb files containing all codes used for data engineering and analysis is included in the total submission package and is availble in our github link

Import the df_ml.csv dataset

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
from pyspark.sql.functions import monotonically_increasing_id

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

import re

### Load Source Data
Now load the df_ml data into a DataFrame. This data consists of reviews(column = "text") that have been previously captured and classified as positive or negative.

Read csv file from DBFS (Databricks File Systems)

## follow the direction to read your table after upload it to Data at the left frame
NOTE: See above for the data type - 

After df_ml_csv file is added to the data of the left frame, create a table using the UI, especially, "Upload File"
tick header and infer schema before creating table

In [8]:
IS_SPARK_SUBMIT_CLI = True

if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [9]:
%fs ls /FileStore/tables/df_ml.csv

path,name,size
dbfs:/FileStore/tables/df_ml.csv,df_ml.csv,77730282


In [10]:
if IS_SPARK_SUBMIT_CLI:
   df_ml = spark.read.csv('df_ml.csv', inferSchema=True, header=True)
else:
    df_ml = spark.sql("SELECT * FROM df_ml_csv")

In [11]:
df_ml.show(5)

+---+-------+--------------------+-------------------+---------+-----------+-----+----+------+-----+
_c0|user_id| text| date|review_id|business_id|funny|cool|useful|stars|
+---+-------+--------------------+-------------------+---------+-----------+-----+----+------+-----+
 0| 21172|Total bill for th...|2013-05-07 04:34:36| 99203| 14562| 1| 0| 6| 1.0|
 1| 74272|I *adore* Travis ...|2017-01-14 21:30:33| 72213| 13614| 0| 0| 0| 5.0|
 2| 67465|I have to say tha...|2016-11-09 20:09:03| 47626| 6862| 0| 0| 3| 5.0|
 3| 32162|Went in for a lun...|2018-01-09 20:56:38| 39301| 12636| 0| 0| 0| 5.0|
 4| 32430|Today was my seco...|2018-01-30 23:07:38| 23694| 852| 0| 0| 7| 1.0|
+---+-------+--------------------+-------------------+---------+-----------+-----+----+------+-----+
only showing top 5 rows

### Prepare the Data
The features for the classification model will be derived from the review text. The label is the stars (stars > 2 = 1 (positive review) else: 0 (negative review)

In [13]:
data = df_ml.select("text", ((col("stars") > 2).cast("Double").alias("label")))
# data = csv
data.show(5)

+--------------------+-----+
 text|label|
+--------------------+-----+
Total bill for th...| 0.0|
I *adore* Travis ...| 1.0|
I have to say tha...| 1.0|
Went in for a lun...| 1.0|
Today was my seco...| 0.0|
+--------------------+-----+
only showing top 5 rows

### Split the Data
In common with most classification modeling processes, we splitted the data into a set for training, and a set for testing the trained model.

In [15]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 83824 Testing Rows: 36176

### Define the Pipeline
The pipeline for the model consist of the following stages:
- A Tokenizer to split the tweets into individual words.
- A StopWordsRemover to remove common words such as "a" or "the" that have little predictive value.
- A HashingTF class to generate numeric vectors from the text values.
- A LogisticRegression algorithm to train a binary classification model.

In [17]:
# convert sentence to words' list
tokenizer = Tokenizer(inputCol="text", outputCol="SentimentWords")
# remove stop words
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="MeaningfulWords")
# convert word to number as word frequency
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
# set the model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.01)

# process pipeline with the series of transforms - 4 transforms
pipeline = Pipeline(stages=[tokenizer, swr, hashTF, lr])

### Run the Pipeline as an Estimator
The pipeline itself is an estimator, and so it has a **fit** method that we called to run the pipeline on a specified DataFrame. In this case, we ran the pipeline on the training data to train a model.

In [19]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")

Pipeline complete!

### Test the Pipeline Model
The model produced by the pipeline is a transformer that will apply all of the stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, we transformed the **test** DataFrame using the pipeline to generate label predictions.

In [21]:
prediction = piplineModel.transform(test)
predicted = prediction.select("text", "prediction", "trueLabel")
predicted.show(10)

+--------------------+----------+---------+
 text|prediction|trueLabel|
+--------------------+----------+---------+
#1 it should be a...| 1.0| 0.0|
$1 oysters everyd...| 1.0| 1.0|
$79 dollars for a...| 1.0| 0.0|
* 4.5 stars \n\nI...| 1.0| 1.0|
*****************...| 1.0| 1.0|
*****************...| 0.0| 0.0|
**I have amended ...| 1.0| 1.0|
**Maybe a 2 star,...| 0.0| 0.0|
**UPDATE**\nWashe...| 0.0| 0.0|
--LOOK AT MY PICT...| 1.0| 0.0|
+--------------------+----------+---------+
only showing top 10 rows

In [22]:
predicted10 = prediction.select("*")
predicted10.show(10)

+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 text|trueLabel| SentimentWords| MeaningfulWords| features| rawPrediction| probability|prediction|
+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
#1 it should be a...| 0.0|[#1, it, should, ...|[#1, advertised, ...|(262144,[14,2433,...|[-2.5650975052305...|[0.07141874592488...| 1.0|
$1 oysters everyd...| 1.0|[$1, oysters, eve...|[$1, oysters, eve...|(262144,[16422,17...|[-6.6968567796218...|[0.00123326419840...| 1.0|
$79 dollars for a...| 0.0|[$79, dollars, fo...|[$79, dollars, ai...|(262144,[14868,21...|[-1.2453808220855...|[0.22350076473034...| 1.0|
* 4.5 stars \n\nI...| 1.0|[*, 4.5, stars, \...|[*, 4.5, stars, \...|(262144,[14,78,47...|[-17.852244869067...|[1.76550339125282...| 1.0|
*****************...| 1.0|[****************...|[****************...|(262144,[3008,518...|[-16.773881086590...|[5.19034207525053...| 1.0|
*****************...| 0.0|[****************...|[****************...|(262144,[19939,21...|[10.7361021928085...|[0.99997825493968...| 0.0|
**I have amended ...| 1.0|[**i, have, amend...|[**i, amended, re...|(262144,[24113,26...|[-3.6126032785629...|[0.02627263934820...| 1.0|
**Maybe a 2 star,...| 0.0|[**maybe, a, 2, s...|[**maybe, 2, star...|(262144,[14,3189,...|[5.44736291364238...|[0.99571082688474...| 0.0|
**UPDATE**\nWashe...| 0.0|[**update**\nwash...|[**update**\nwash...|(262144,[14,2841,...|[17.0731875012253...|[0.99999996152232...| 0.0|
--LOOK AT MY PICT...| 0.0|[--look, at, my, ...|[--look, picture!...|(262144,[14,23192...|[-0.6135929202065...|[0.35124004054298...| 1.0|
+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

##TP, FP, TN, and FN all calculated
Precision and recall also calculated

In [24]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
      ("TP", tp),
      ("FP", fp),
      ("TN", tn),
      ("FN", fn),
      ("Precision", tp / (tp + fp)),
      ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()

+---------+------------------+
 metric| value|
+---------+------------------+
 TP| 26711.0|
 FP| 2577.0|
 TN| 5671.0|
 FN| 1217.0|
Precision|0.9120117454247474|
 Recall|0.9564236608421656|
+---------+------------------+

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR =", aur)

AUR = 0.8219921407993562

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## AUC is calculated

In [29]:
gbt_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
gbt_auc = gbt_evaluator.evaluate(prediction)

print("AUC for Logistic Regression = ", gbt_auc)

AUC for Logistic Regression = 0.8916586030414259

## Generated AUC for Logistic Regression =  0.8916586030414259